### Imports

In [159]:
import pandas as pd
import pickle
from scipy.stats import poisson

### Data Cleaning and Preparation

In [147]:
df_historical = pd.read_csv('data/afcon_historical_data.csv')
df_fixture = pd.read_csv('data/afcon_fixture_data.csv')

##### Cleanup Fixture Data

In [148]:
df_fixture["home"] = df_fixture["home"].str.strip()
df_fixture["away"] = df_fixture["away"].str.strip()

In [149]:
df_fixture.drop(columns=["score"], axis=1, inplace=True)

In [150]:
df_fixture.rename(columns={"home": "home_team", "away": "away_team"}, inplace=True)

##### Cleanup Historical Data

In [151]:
df_historical.dropna(inplace=True)
df_historical.drop_duplicates(inplace=True)
df_historical.sort_values(by="year", inplace=True)

In [152]:
# remove cancelled games from historical data
df_historical = df_historical[df_historical["score"] != "Cancelled"]

In [153]:
# split score into home and away goals
df_historical[["home_goals", "away_goals"]] = df_historical["score"].str.split("–", expand=True)
df_historical = df_historical.astype({"home_goals": "int64", "away_goals": "int64"})
df_historical.drop(columns=["score"], axis=1, inplace=True)

In [154]:
df_historical.rename(columns={"home": "home_team", "away": "away_team"}, inplace=True)

In [155]:
df_historical["total_goals"] = df_historical["home_goals"] + df_historical["away_goals"]

#### Export Clean Data

In [156]:
df_historical.to_csv("data/afcon_historical_data_clean.csv", index=False)
df_historical.to_csv("data/afcon_fixture_data_clean.csv", index=False)

In [157]:
df_historical

,home_team,away_team,year,home_goals,away_goals,total_goals
0,Sudan,Egypt,1957,1,2,3
1,Ethiopia,South Africa,1957,2,0,2
2,Egypt,Ethiopia,1957,4,0,4
3,United Arab Republic,Ethiopia,1959,4,0,4
4,Sudan,Ethiopia,1959,1,0,1
...,...,...,...,...,...,...
714,Nigeria,Sudan,2021,3,1,4
715,Guinea-Bissau,Egypt,2021,0,1,1
716,Guinea-Bissau,Nigeria,2021,0,2,2
718,Algeria,Sierra Leone,2021,0,0,0


In [158]:
df_fixture.sample(10)

,home_team,away_team,year
31,DR Congo,Zambia,2023
15,Guinea,Gambia,2023
8,Egypt,Ghana,2023
43,Winner Group F,Runner-up Group E,2023
39,Runner-up Group B,Runner-up Group F,2023
32,Morocco,DR Congo,2023
10,Mozambique,Ghana,2023
9,Cape Verde,Mozambique,2023
33,Zambia,Tanzania,2023
48,Winner QF1,Winner QF4,2023


### Prediction Model

In [ ]:
# Egypt AFCON 2023